In [3]:
from __future__ import division
from astropy.io import fits
import numpy as np
from astropy.cosmology import FlatLambdaCDM
from astropy import units as u
from astropy.table import Table
import math
import csv

H0 = 72.
cosmo = FlatLambdaCDM(H0, Om0=0.3)
h = H0/100

In [3]:
hdulist = fits.open('../JHU_mstell_ssfr_Yang_gzoo.fits')
tbdata = hdulist[1].data
color = tbdata['col10'] #^{0.1}(g-r), K-E corrected to z=0.1, petro
M_r = tbdata['col9'] #^{0.1}(M_r) - 5\log h, K-E corrected to z=0.1, petro
centsat = tbdata['col4_2'] #if BRIGHTEST galaxy in group, is 1, otherwise 2
pfeatures = tbdata['t01_smooth_or_features_a02_features_or_disk_weighted_fraction']
psmooth = tbdata['t01_smooth_or_features_a01_smooth_weighted_fraction']
#absmag = tbdata['ABSMAG']
#absmag_r = absmag[:,4]
ssfr = tbdata['AVG_SSFR']
z = tbdata['Z'] #Schlegel redshift from mpa-jhu
z_warn = tbdata['Z_WARNING'] #if non-zero, don't sue Schlegel redshift
#mass = tbdata['MASS'] #stellar mass from K correction fit, in h^-2 Msun units (from nsatlas)
mass = tbdata['AVG_MSTELL']
group_num = tbdata['col3_2']
galaxy_num = tbdata['col1_2']
hdulist.close()

hdulist2 = fits.open('../petroC_group.fits')
tbdata2 = hdulist2[1].data
petroC_group_num = tbdata2['col1'] #group number
petroC_halo_mass = tbdata2['col7'] #Luminosity weighted halo mass estimate, units: log M_halo/ (M_{\odot}/h)
hdulist2.close()

In [5]:
#Magnitude limit calculation
z_lim = 0.06
d = cosmo.luminosity_distance(z_lim).to(u.pc)
m = 17
M_limit = m - 5*math.log(d.value,10) + 5 
adj_M_limit = M_limit - 5*math.log(h,10)
print(M_limit, adj_M_limit)

(-20.084842305774437, -19.37150478793078)


In [6]:
halo_mass = []

redpas_ind = []
redpas_pfeatures = []
redpas_psmooth = []
redpas_color = []
redpas_sfr = []
redpas_mr = []
redpas_mass = []
redpas_galnum = []

bluact_ind = []
bluact_pfeatures = []
bluact_psmooth = []
bluact_color = []
bluact_sfr = []
bluact_mr = []
bluact_mass = []
bluact_galnum = []

redact_ind = []
redact_pfeatures = []
redact_psmooth = []
redact_color = []
redact_sfr = []
redact_mr = []
redact_mass = []
redact_galnum = []

group_list = [[67926]]

for i in np.arange(pfeatures.size):
    group_ind = np.where(petroC_group_num == group_num[i])
    halo_mass.append(petroC_halo_mass[group_ind[0]])
    
    if z[i] > 0.01 and z[i] < 0.06 and M_r[i] < adj_M_limit:
        groups = np.array([item[0] for item in group_list])
        ind = np.where(groups == group_num[i])
        if ind[0].size == 0: #if the group isn't in the array yet
            group_list.append([group_num[i]])
            group_list[-1].append(galaxy_num[i])
        else:
            group_list[np.squeeze(ind[0])].append(galaxy_num[i])
        color_line = 0.7 - 0.032*(M_r[i] + 16.5)
        active_line = -10 + 0.094*(M_r[i] + 15)
        logSSFR = ssfr[i]
        
        if color[i] > color_line: #red galaxies
            if logSSFR <= active_line: #passive galaxies - redpas type
                redpas_ind.append((i, centsat[i]))
                redpas_pfeatures.append((pfeatures[i], centsat[i]))
                redpas_psmooth.append((psmooth[i], centsat[i]))
                redpas_color.append((color[i], centsat[i]))
                redpas_sfr.append((logSSFR, centsat[i]))
                redpas_mr.append((M_r[i], centsat[i]))
                redpas_mass.append((mass[i], centsat[i]))
                redpas_galnum.append((galaxy_num[i], centsat[i]))
            else: #red and active - redact type
                redact_ind.append((i, centsat[i]))
                redact_pfeatures.append((pfeatures[i], centsat[i]))
                redact_psmooth.append((psmooth[i], centsat[i]))
                redact_color.append((color[i], centsat[i]))
                redact_sfr.append((logSSFR,centsat[i]))
                redact_mr.append((M_r[i],centsat[i]))
                redact_mass.append((mass[i], centsat[i]))
                redact_galnum.append((galaxy_num[i], centsat[i]))
        elif logSSFR >= active_line and color[i] < color_line: #blue and active - bluact type
            bluact_ind.append((i, centsat[i]))
            bluact_pfeatures.append((pfeatures[i],centsat[i]))
            bluact_psmooth.append((psmooth[i],centsat[i]))
            bluact_color.append((color[i],centsat[i]))
            bluact_sfr.append((logSSFR,centsat[i]))
            bluact_mr.append((M_r[i],centsat[i]))
            bluact_mass.append((mass[i],centsat[i]))
            bluact_galnum.append((galaxy_num[i], centsat[i]))

In [20]:
group_list_arr = np.array([np.array(xi) for xi in group_list])

In [17]:
tredpas = Table([redpas_ind, redpas_pfeatures, redpas_psmooth, redpas_color, redpas_sfr, redpas_mr, redpas_mass, redpas_galnum], names = ('redpas_ind', 'redpas_pfeatures', 'redpas_psmooth', 'redpas_color', 'redpas_sfr', 'redpas_mr', 'redpas_mass', 'redpas_galnum'))
tredpas.write('../lookup_redpas_cut.fits', format='fits')

tbluact = Table([bluact_ind, bluact_pfeatures, bluact_psmooth, bluact_color, bluact_sfr, bluact_mr, bluact_mass, bluact_galnum], names = ('bluact_ind', 'bluact_pfeatures', 'bluact_psmooth', 'bluact_color', 'bluact_sfr', 'bluact_mr', 'bluact_mass', 'bluact_galnum'))
tbluact.write('../lookup_bluact_cut.fits', format='fits')

tredact = Table([redact_ind, redact_pfeatures, redact_psmooth, redact_color, redact_sfr, redact_mr, redact_mass, redact_galnum], names = ('redact_ind', 'redact_pfeatures', 'redact_psmooth', 'redact_color', 'redact_sfr', 'redact_mr', 'redact_mass', 'redact_galnum'))
tredact.write('../lookup_redact_cut.fits', format='fits')

alltab = Table([halo_mass, pfeatures, psmooth, color, ssfr, M_r, mass, z, centsat, galaxy_num, group_num], names = ('halo_mass', 'pfeatures', 'psmooth', 'color', 'ssfr', 'M_r', 'mass', 'z', 'centsat', 'galaxy_num', 'group_num'))
alltab.write('../lookup_all_cut.fits', format='fits')

ValueError: Illegal format ('O', (2,)).

In [28]:
with open("lookup_group_cut.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(group_list)

In [18]:
alltab

halo_mass [2],pfeatures,psmooth,color,ssfr,M_r,mass,z,centsat,galaxy_num,group_num
object,float32,float32,float64,float32,float64,float32,float32,int64,int64,int64
[ 12.3163] .. 2,0.137,0.824,0.567812,-10.116,-19.802763,10.1547,0.0650235,2,25060,25112
[ 12.8741] .. 1,0.087,0.87,0.9982677,-11.8113,-21.3413029,11.1778,0.121316,1,25064,84512
[ 12.3163] .. 1,0.191,0.787,0.8753343,-11.5204,-19.922863,10.5777,0.0645888,1,25061,25112
[ 12.2007] .. 1,1.0,0.0,0.7734479,-10.466,-20.3655033,10.655,0.0480963,1,25503,84808
[ 12.1431] .. 1,0.064,0.894,0.9542314,-12.1338,-20.3300915,10.8579,0.0841035,1,10134,74674
[ 12.263] .. 1,0.447,0.553,0.8453194,-11.236,-20.5215645,11.0639,0.0902407,1,8736,73726
[ 12.158] .. 1,0.366,0.634,0.7352224,-10.1443,-20.3526897,10.6943,0.0652254,1,7209,72748
[ 12.5087] .. 1,1.0,0.0,0.6250712,-9.95137,-20.9211197,10.8847,0.0649208,1,7210,72749
[ 11.8219] .. 1,0.109,0.87,0.5931152,-9.94026,-19.6553574,10.1631,0.065152,1,8732,73724
